In [ ]:
import numpy as np

BOARD_SIZE = (56, 56)
board = np.zeros(BOARD_SIZE, dtype=int)
block_list = [(4,14),(6,28),(7,10),(7,28),(10,32),(11,32),(14,17),(14,21),(14,21),(14,28),(18,21),(18,21)]

In [ ]:
blocks = [(i+1, b) for i, b in enumerate(block_list)] # Add unique indexing, Reserve 0 for not placed
blocks

In [ ]:
# Available space from place_coords for adding a new block
def get_avail_space(board, place_coords):
    r, c = place_coords
    rcnt = 0
    for v in board[r:,c]:
        if v > 0:
            break
        rcnt += 1
        
    ccnt = 0
    for v in board[r,c:]:
        if v > 0:
            break
        ccnt += 1
    return (rcnt, ccnt)

# Placed block is not over the board edge
def block_not_off_board(board, place_coords, block):
    r, c = place_coords
    block_r, block_c = block
    
    # Space Available
    avail_r = board.shape[0] - r
    avail_c = board.shape[1] - c    
    
    if block_r <= avail_r and block_c <= avail_c:
        return True
    else:
        return False

# Sliver is a gap small enough that no block fits in to it
def no_sliver(board, place_coords, min_block_width):
    r, c = place_coords
    result = np.where(np.sum(board[r:, c:], axis=0) > 0)[0]
    if result.size != 0:
        size = result[0]
    
    gap_r = np.sum(np.where(np.sum(board, axis=1) == 0, 1, 0))
    gap_c = np.sum(np.where(np.sum(board, axis=0) == 0, 1, 0))
    
    for gap in [gap_r, gap_c]:
        if gap < min_block_width and gap > 0:
            return False
    return True    

# Adds block index to board array if it doesn't go off the board edge and doesn't overlap another block
def add_block_to_board(board, place_coords, block, val=1):
    if block_not_off_board(board, place_coords, block):
        r, c = place_coords
        
        block_temp = np.zeros(board.shape, dtype=int)
        block_temp[r:r+block[0], c:c+block[1]] = val

        # Make sure block doesn't overlap with already placed blocks
        if np.max(np.where(board > 0, 1, 0) + np.where(block_temp > 0, 1, 0)) == 1:
            return 'None', board + block_temp
    return 'Error', board

# Remove block index from board array
def remove_block_from_board(board, val):
    return np.where(board == val, 0, board)

# All blocks placed
def board_is_completed(board):
    if np.min(board) == 0:
        return False
    else:
        return True
    
# Of the blocks still available for placement, get the smallest possible width
def get_min_block_width(blocks):
    return min([min(b) for _, b in blocks])     

In [ ]:
# Get the row and column coordinates of the first 0 value in the np array
def get_next_place(board, block, min_block_width):
    error = 'None'
    
    # Get the coordinates of first 0
    r, c = np.unravel_index(np.argmin(board), board.shape)
    
    # Slivers are regions that are too small for any available block to fit
    if not no_sliver(board, (r,c), min_block_width):
        return 'Error', (r, c)

    error, _ = add_block_to_board(board, (r, c), block)
    if error == 'None':
        return error, (r, c)
    
    return 'Error', (r, c)

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline

# Arbitrary colors for display purposes
color_dict = {0: (255, 255, 255),
             1: (235, 64, 52),
             2: (235, 217, 52),
             3: (189, 235, 52),
             4: (52, 177, 235),
             5: (192, 52, 235),
             6: (235, 52, 195),
             7: (103, 80, 52),
             8: (235, 162, 52),
             9: (32, 171, 39),
             10: (14, 54, 135),
             11: (135, 14, 125),
             12: (135, 14, 58)}

# Display board as matplotlib image
def pretty_print_board(board, init_clear=True):
    if init_clear:
        clear_output(wait=True)
    
    h, w = board.shape
    board_rgb = np.zeros((*board.shape, 3), dtype=int)
    for r in range(board.shape[0]):
        for c in range(board.shape[1]):
            board_rgb[r, c, :] = color_dict[board[r, c]]
    
    plt.imshow(board_rgb, interpolation='nearest')
    plt.show()

# Main Recursive Loop

In [ ]:
def calibron(board, blocks, solutions):
    rotations = [(idx, block[::-1]) for idx, block in blocks]
    for i, (idx, block) in enumerate(blocks + rotations):
        if board_is_completed(board):
            solutions += [board]
            pretty_print_board(board)
            input('Hit Enter to continue.')
            return
            
        if len([(ix, bl) for ix, bl in blocks if ix not in board]) == 0:
            return

        if idx in board:
            continue
        
        min_block_width = get_min_block_width([(ix, bl) for ix, bl in blocks if ix not in board])
        error, next_place = get_next_place(board, block, min_block_width)
        if error != 'Error':
            place_r, place_c = next_place
            error, board = add_block_to_board(board, (place_r, place_c), block, val=idx)
            if error != 'None':
                print('Error adding block to board')
                continue

            calibron(board, [(ix, bl) for ix, bl in blocks if ix not in board and ix != idx], solutions)

            if not(board_is_completed(board)):
                board = remove_block_from_board(board, idx)
    return

# Run Solver

In [ ]:
solutions = []
calibron(board, blocks, solutions)

In [ ]:
len(solutions)

# Unique Solutions

In [ ]:
def board_is_unique(new_board, board_list):
    for k in range(4):
        board_test = np.rot90(new_board, k=k)
        if any([np.array_equal(board_test, b) for b in board_list]):
            return False
        elif any([np.array_equal(np.flip(board_test, axis=0), b) for b in board_list]):
            return False
        elif any([np.array_equal(np.flip(board_test, axis=1), b) for b in board_list]):
            return False
        elif any([np.array_equal(np.flip(board_test, axis=[0,1]), b) for b in board_list]):
            return False
    return True

In [ ]:
unique = []
for solution in solutions:
    if board_is_unique(solution, unique):
        unique += [solution]
print(len(unique))

In [ ]:
for u in unique:
    pretty_print_board(u, init_clear=False)